# Basic RAG

High-level overview of the system we want to build, but in this case this won't be youtube video, but simple text about family.


![lol](../images/system1.png)

Let's start program by loading environment variables

In [14]:
import os
from dotenv import load_dotenv

In [15]:
load_dotenv()

def assert_env_key(key: str | None, name: str) -> None:
    if not key:
        raise ValueError(f"{name} is not set, please update .env file")    

OPENAI_API_KEY_NAME = "OPENAI_API_KEY"
OPENAI_API_KEY = os.getenv(OPENAI_API_KEY_NAME)
assert_env_key(OPENAI_API_KEY, OPENAI_API_KEY_NAME)

## Setting up the model

Define the LLM model that will be used as part of workflow.

In [16]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(model = "gpt-3.5-turbo", api_key=OPENAI_API_KEY)

### Ensure that model works correctly

In [ ]:
model.invoke("How much is 2 + 2?")

AIMessage(content='The Los Angeles Dodgers won the World Series during the COVID-19 pandemic. They defeated the Tampa Bay Rays in the 2020 World Series to claim their first championship since 1988.')

The result from the model is an AIMessage instance containing the answer. We can extract this answer by chaining the model with an [output parser](https://python.langchain.com/docs/how_to/#output-parsers).

Here is what chaining the model with an output parser looks like:

![Chain1](../images/chain1.png)

For this example, we'll use a simple `StrOutputParser` to extract the answer as a string.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("What MLB team won the World Series during the COVID-19 pandemic?")

Output is: 'The Los Angeles Dodgers won the World Series during the COVID-19 pandemic in 2020.'

### Introducing prompt templates

We want to provide the model with some context and the question. [Prompt templates](https://python.langchain.com/docs/how_to/#prompt-templates) are a simple way to define and reuse prompts.

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Mary's sister is Susana", question="Who is Mary's sister?")

We can now chain the prompt with the model and the output parser.

![](../images/chain2.png)

In [ ]:
chain = prompt | model | parser
chain.invoke({
    "context": "Mary's sister is Susana",
    "question": "Who is Mary's sister?"
})

### Combining chains
We can combine different chains to create more complex workflows. For example, let's create a second chain that translates the answer from the first chain into a different language.

Let's start by creating a new prompt template for the translation chain:

In [ ]:
translation_prompt = ChatPromptTemplate.from_template(
    "Translate {answer} to {language}"
)

We can now create a new translation chain that combines the result from the first chain with the translation prompt.

Here is what the new workflow looks like:
![](../images/chain3.png)

In [ ]:
from operator import itemgetter

translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.invoke(
    {
        "context": "Mary's sister is Susana. She doesn't have any more siblings.",
        "question": "How many sisters does Mary have?",
        "language": "Spanish",
    }
)